Using Keras to solve the Kaggle competition: House Prices: Advanced Regression Techniques  
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print('train data shape:',train_data.shape)
print('test data shape:', test_data.shape)

train data shape: (1460, 81)
test data shape: (1459, 80)


In [6]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


First we need to concat the training data and the test data to gether since we need to apply the feature engineering simultaneously.

In [3]:
all_data = pd.concat((train_data.iloc[:,1:-1],test_data))

Check the missing value.

In [4]:
all_data_na = all_data.isnull().sum()
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index)
all_data_na = all_data_na.sort_values(ascending = False)
na_data = pd.DataFrame({'Na Value':all_data_na})
na_data

,Na Value
PoolQC,2909
MiscFeature,2814
Alley,2721
Fence,2348
Id,1460
FireplaceQu,1420
LotFrontage,486
GarageCond,159
GarageFinish,159
GarageQual,159


We still need to do the normalization, and simpy fill the NA values with the mean

In [5]:
numeric_features = all_data.dtypes[all_data.dtypes != 'object'].index
all_data[numeric_features] = all_data[numeric_features].apply(
lambda x:(x-x.mean())/x.std())

all_data = all_data.fillna(all_data.mean())

all_data = pd.get_dummies(all_data,dummy_na = True)

print('The shape of all data set: ',all_data.shape)

The shape of all data set:  (2919, 332)


Separate the training set and the test set.

In [8]:
ntrain = train_data.shape[0]
train_set = np.array(all_data[:ntrain].values)
test_set = np.array(all_data[ntrain:].values)
train_labels = np.array(train_data.SalePrice.values).reshape(-1,1)

Build a simple model to train it.

In [18]:
from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Model
from keras import regularizers

import keras.backend as K

In [37]:
def simplemodel(input_shape):
    X_input=Input(input_shape)
    X=Dense(256,activation='relu')(X_input)
    X=Dense(128,activation='relu')(X)
    X=Dense(1)(X)
    model = Model(inputs = X_input, outputs = X, name='simple')
    return model

In [38]:
Simplemodel = simplemodel(train_set.shape[1:])
Simplemodel.summary

<bound method Network.summary of <keras.engine.training.Model object at 0x00000245B1765B70>>

In [39]:
Simplemodel.compile(optimizer = 'Adam', loss = "mean_squared_error",metrics=["mean_squared_error"])

In [40]:
Simplemodel.fit(x = train_set, y = train_labels,epochs = 100, batch_size = 64,verbose=1)

Epoch 1/100
1460/1460 [==============================] - 0s 327us/step - loss: 39034718000.3945 - mean_squared_error: 39034718000.3945
Epoch 2/100
1460/1460 [==============================] - 0s 94us/step - loss: 38993956089.6877 - mean_squared_error: 38993956089.6877
Epoch 3/100
1460/1460 [==============================] - 0s 88us/step - loss: 38831201832.6795 - mean_squared_error: 38831201832.6795
Epoch 4/100
1460/1460 [==============================] - 0s 93us/step - loss: 38405837327.4301 - mean_squared_error: 38405837327.4301
Epoch 5/100
1460/1460 [==============================] - 0s 89us/step - loss: 37542794293.3041 - mean_squared_error: 37542794293.3041
Epoch 6/100
1460/1460 [==============================] - 0s 86us/step - loss: 36078523961.5123 - mean_squared_error: 36078523961.5123
Epoch 7/100
1460/1460 [==============================] - 0s 89us/step - loss: 33853344119.9342 - mean_squared_error: 33853344119.9342
Epoch 8/100
1460/1460 [==============================] - 0s 9

1460/1460 [==============================] - 0s 92us/step - loss: 1034161862.8384 - mean_squared_error: 1034161862.8384
Epoch 63/100
1460/1460 [==============================] - 0s 88us/step - loss: 1026706626.8055 - mean_squared_error: 1026706626.8055
Epoch 64/100
1460/1460 [==============================] - 0s 87us/step - loss: 1021906092.0110 - mean_squared_error: 1021906092.0110
Epoch 65/100
1460/1460 [==============================] - 0s 90us/step - loss: 1015847829.3918 - mean_squared_error: 1015847829.3918
Epoch 66/100
1460/1460 [==============================] - 0s 89us/step - loss: 1010786342.4000 - mean_squared_error: 1010786342.4000
Epoch 67/100
1460/1460 [==============================] - 0s 87us/step - loss: 1007193354.3452 - mean_squared_error: 1007193354.3452
Epoch 68/100
1460/1460 [==============================] - 0s 88us/step - loss: 998699620.6466 - mean_squared_error: 998699620.6466
Epoch 69/100
1460/1460 [==============================] - 0s 90us/step - loss: 99317

Use the evaluation method provided by the competition:

In [41]:
def rmlse2(y_true, y_pred):
    a = np.mean(np.square((np.log(y_pred)-np.log(y_true))**2))
    return a

In [42]:
pred = Simplemodel.predict(train_set)
print(rmlse2(pred, train_labels))

0.00509644230855


In [43]:
pred_test=Simplemodel.predict(test_set)
test_data['SalePrice'] = pd.Series(pred_test.reshape(1, -1)[0])
submission_nn = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
submission_nn.to_csv('submission_nn.csv', index=False)

Upload the prediction and see the score.  
scored 0.15246.

The result is not good enough, which means there is more work to be done.